-----------------------
## 1) New way of scraping  11.4.2024
-----------------------

In [ ]:
from datetime import datetime
import pandas as pd
from tqdm import tqdm  
import requests 

import os
os.chdir("c:\\Users\\jirka\\Documents\\MyProjects\\Sreality")
from scraper.scraper import SrealityScraper
from db_managment.data_manager import DataManager
from utils.utils import Utilities
from run import Runner

### My Simple run - no follow-up scraping

In [ ]:
scraper = SrealityScraper()
utils = Utilities()

full_datetime, date_to_save = utils.generate_timestamp()
print(full_datetime, date_to_save)

data_prodej_byty = scraper.scrape_all_with_filter(timestamp=full_datetime, category_main_cb=1,category_type_cb=1)
df_data_prodej_byty = pd.DataFrame(data_prodej_byty)
utils.safe_save_csv(df_data_prodej_byty, f"data_{date_to_save}")

data_all = scraper.scrape_all_with_filter(timestamp=full_datetime)
df_data_all = pd.DataFrame(data_all)
utils.safe_save_csv(df_data_all, f"data_all_{date_to_save}")

In [ ]:
# Just testing count
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/count"
    result =  session.request(method="GET", url=url, headers=headers).json()
    print(result)
    url_filter = f"https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=1&page=1"
    result2 =  session.request(method="GET", url=url_filter).json()
    print(result2)

In [ ]:
scraper = SrealityScraper()
utils = Utilities()
full_datetime, date_to_save = utils.generate_timestamp()

new_data = scraper.scrape_specific_estates(["95413580"], full_datetime)
display(new_data)

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/95413580"
    result =  session.request(method="GET", url=url, headers=headers).json()
    display(result)


## Full Run with follow-up scraping

In [ ]:
runner = Runner()

res = runner.regular_run(scrape_prodej_byty=True, scrape_all=True)

## Test Utils

In [1]:
from db_managment.data_manager import DataManager
from utils.utils import Utilities
import pandas as pd

data_manager = DataManager()
utils = Utilities()

In [2]:
df_new = utils.prep_df_new_estates("20240422_1647.json")
print(df_new.columns)
df_new.tail(2)

Index(['code', 'description', 'meta_description', 'category_main_cb',
       'category_sub_cb', 'category_type_cb', 'broker_id', 'broker_company',
       'furnished', 'locality_gps_lat', 'locality_gps_lon', 'object_type',
       'parking_lots', 'locality_street_id', 'locality_district_id',
       'locality_ward_id', 'locality_region_id', 'locality_quarter_id',
       'locality_municipality_id', 'locality_country_id', 'terrace', 'balcony',
       'loggia', 'basin', 'cellar', 'building_type', 'object_kind',
       'usable_area', 'ownership', 'low_energy', 'easy_access',
       'building_condition', 'garage', 'room_count_cb',
       'energy_efficiency_rating_cb', 'note_about_price', 'last_update',
       'material', 'age_of_building', 'ownership_type', 'floor', 'floor_area',
       'no_barriers', 'timestamp', 'id_of_order', 'start_of_offer'],
      dtype='object')


,code,description,meta_description,category_main_cb,category_sub_cb,category_type_cb,broker_id,broker_company,furnished,locality_gps_lat,...,last_update,material,age_of_building,ownership_type,floor,floor_area,no_barriers,timestamp,id_of_order,start_of_offer
53,2550375756,S radostí Vám představujeme výjimečný byt 3+kk...,"Byt 3+kk 79 m² k prodeji Holýšovská, Praha 5 -...",1.0,6.0,1.0,84338,BVS INVESTMENTS s.r.o.,2.0,50.041056,...,07.03.2024,Smíšená,Velmi dobrý,Osobní,10. podlaží,79,NaN,2024-04-22 16:38:39,000022,NaN
54,2878338380,Nabízíme k prodeji byt o velikosti 3 + 1 v oso...,"Byt 3+1 102 m² k prodeji Moučná, Poděbrady - P...",1.0,7.0,1.0,287156,Honzejková,3.0,50.150831,...,21.03.2024,Smíšená,Velmi dobrý,Osobní,1. podlaží z celkem 5,102,NaN,2024-04-22 16:38:39,0184,NaN


In [3]:
timestamp, _ = utils.generate_timestamp()
#TODO: musím nejprve vygenerovat 4 sloupečky žejo, to locality, city ,district, region
#TODO: naopak missing vlaues jsou asi OK, prostě mám NANy v df ..
#TODO.. jo aha to není řešení. pokud value chybí u všech položek v JSONu, tak sloupec zmizí a tradá.

data_manager._insert_new_estate(df_new, timestamp)

KeyError: 'locality'

## Table managment test

In [ ]:
from db_managment.data_manager import DataManager
import pandas as pd

data_manager = DataManager()

In [ ]:
data_manager.get_all_rows("estate_detail")

In [ ]:
data_manager.clear_table("estate_detail")

In [ ]:
data_manager.drop_table("estate_detail")

In [ ]:
data_manager.create_table("estate_detail")

In [ ]:
upload = pd.DataFrame(data = [[42, "Prodej luxsního bejváku", "Byt 14+kk daleko od centra, ale v klidu.",
                               1, 6, 1, 111, "M&M reality", 2, 49.827491575, 18.1654443,
                               0, 0, 111083, 65, 14829, 12, 10, 4730, 112,
                               0, 0, 0, 0, 0, 2, 0, 79, "Družstevní", 0, 1, 1, 0,
                               3, 0, 870824, 1, 2,
                               ### Not mandatory:
                               "Cena je finální", 123,
                               "Dnes", "cihla", "Novostavba", "Družstevní", "2.patro",
                               42, 43, 2, "true"
                               ]],
                      columns=['code', 'description', 'meta_description', 
                               'category_main_cb', 'category_type_cb', 'category_sub_cb', 'broker_id', 'broker_company',
                            'furnished', 'latitude', 'longitude', 'locality', 'city', 'district',
                            'region', 'object_type', 'parking_lots', 'locality_street_id',
                            'locality_district_id', 'locality_ward_id', 'locality_region_id',
                            'locality_quarter_id', 'locality_municipality_id',
                            'locality_country_id', 'terrace', 'balcony', 'loggia', 'basin',
                            'cellar', 'building_type', 'object_kind', 'ownership', 'low_energy',
                            'easy_access', 'building_condition', 'garage', 'room_count_cb', 'energy_efficiency_rating_cb', 
                            ### Not mandatory:
                            'note_about_price', 'id_of_order',
                            'last_update', 'material', 'age_of_building', 'ownership_type', 'floor',
                            'usable_area', 'floor_area', 'energy_efficiency_rating', "no_barriers"
                            ])

data_manager._insert_new_estate(upload, "2024-04-21 23:22:44")

#TODO: how about unit test for this?
#? should fail predictably as it breakes foreign key constraints i made:)
upload = pd.DataFrame(data = [["911", "4242", "12345000"]],
                      columns=["estate_id", "batch_id", "price"])

data_manager.insert_new_price(upload)